# Exploratory Data Analysis - Crime Data Los Angeles

This notebook is used to explore the questions for the Data Scientist Exercise in July 2020.

- How many crimes were reported over the past 5 years?
- List the top five reported crimes for each year for the past 5 years.
- What are the most common MO codes? Have these changed over the past 5 years?
- How else has reported crime changed over time in the City?
- Based on your analysis, please share any changes to services, programs, or policies that the City should consider.

##  Load libraries

In [10]:
# Import libraries
import os, sys, subprocess
import json
import pandas as pd
import numpy as np
from langdetect import detect
import pickle

pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 500)

In [14]:
# load project config
terminal_call = ! git rev-parse --show-toplevel
repo_path=terminal_call[0]
project_config_path = os.path.join(repo_path,'project_config.json')

with open(project_config_path,'r') as fp: 
    project_config = json.load(fp)

In [25]:
# import custom module to look at trends
module_path = os.path.join(repo_path,project_config['project_module_relative_path'])
sys.path.append(module_path)

import trends
from trends import get_top_trends as gt

from importlib import reload

# Load in data

In [20]:
# load in data that was collected
df = pickle.load(open("../data/crime_data.pkl", "rb"))

## How many crimes were reported over the past 5 years?

In [21]:
# Check how many crimes were reported over the past 5 years
# Calculate current date and offset by 5 years
(df["date_rptd"] > (pd.datetime.now()- pd.DateOffset(years=5))).sum()

1117864

### A: 1,117,864 crimes were reported in the past 5 years

## List the top five reported crimes for each year for the past 5 years.

In [31]:
# Select the last 5 years as a dataframe
df_5 = df[df["date_rptd"] > (pd.datetime.now()- pd.DateOffset(years=5))]

In [35]:
# Find the top 5 reported crimes for each year
# Crime is listed in the data under 'crm_cd_1'
# Custom function 'top_trends' groups by column and selects the top 'n' 
gt.top_trends(df_5, column='year', variable='crm_cd_1', n=5)

year  crm_cd_1
2015  624.0       8866 
      510.0       8573 
      440.0       7863 
      330.0       7319 
      354.0       7300 
2016  510.0       18348
      624.0       17983
      330.0       16784
      440.0       14810
      310.0       14558
2017  624.0       19134
      510.0       18748
      330.0       18097
      310.0       15282
      440.0       14773
2018  624.0       19567
      330.0       18080
      510.0       17124
      440.0       15414
      310.0       14814
2019  624.0       19124
      330.0       16755
      440.0       15453
      510.0       15425
      740.0       12889
2020  510.0       9763 
      624.0       8233 
      330.0       6895 
      310.0       6694 
      740.0       6379 
Name: crm_cd_1, dtype: int64

## What are the most common MO codes? Have these changed over the past 5 years?

In [36]:
# Find the top 10 most common MO codes across all the years (2010-2020)
df_5["mocodes"].value_counts(ascending = False).head(10)

0344         93842
0329         44407
1501         18472
0325         12737
0416         12317
0329 1300    8302 
1822         8234 
0344 1300    6090 
0344 1606    5898 
0329 1307    4808 
Name: mocodes, dtype: int64

In [37]:
# Find the top 10 MO codes for each year
# MO code is listed in the data under 'mocodes'
# Custom function 'top_trends' groups by column and selects the top 'n' 
gt.top_trends(df_5, column='year', variable='mocodes', n=5)

year  mocodes  
2015  0344         12619
      0329         4784 
      1501         2013 
      0325         1632 
      0416         1573 
2016  0344         22719
      0329         9713 
      1501         4215 
      0325         3037 
      0416         2939 
2017  0344         20205
      0329         9811 
      1501         4436 
      0416         2938 
      0325         2737 
2018  0344         17485
      0329         9032 
      1501         3705 
      0416         2375 
      0325         2354 
2019  0344         15849
      0329         8156 
      1501         2984 
      0325         2327 
      0416         1977 
2020  0344         4965 
      0329         2911 
      1501         1119 
      0344 1300    706  
      0329 1300    698  
Name: mocodes, dtype: int64